In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_Full/models/B5R2_15AB_Full_set1_AddNewDataset.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image_13122564/'
dataframe = pd.read_csv( '/home/yupaporn/CSV_file/Train_Dataset_CCA_5300.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/CSV_file/Test_Dataset_CCA_1328.csv')
base_dir0 = '/media/tohn/SSD/Images/Image_13122564/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv( '/home/yupaporn/CSV_file/Test_Dataset_CCA_1328.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(1328, 25)
(1328, 25)
Normal:  (857, 25)
Abnormal:  (471, 25)


,Unnamed: 0,Unnamed: 0.1,Case,Abs Position,Sub Position,Class,Path Full,Path Crop,Views,fold,...,height,Rleft,Rtop,Rwidth,Rheight,Sub_class_New,tag_AjNit,tag_AjWan,Test150,Spilt
0,111,111,AB01_40,P1,P1,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,2,...,311.0,0.092664,0.148873,0.513514,0.346614,AB01,NaN,NaN,False,Test
1,112,112,AB01_40,P2,P2,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,2,...,385.0,0.171698,0.154849,0.560377,0.428287,AB01,NaN,NaN,False,Test
2,113,113,AB01_40,P4,P41,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,2,...,640.0,0.140316,0.150865,0.667984,0.711155,AB01,NaN,NaN,False,Test
3,114,114,AB01_40,P5,P51,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-C,2,...,587.0,0.063462,0.107041,0.690385,0.653386,AB01,NaN,NaN,False,Test
4,115,115,AB01_40,P3,P31,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,2,...,562.0,0.216374,0.146881,0.672515,0.625498,AB01,NaN,NaN,False,Test


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Sub_class_New',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 1328 validated image filenames belonging to 20 classes.
{0: 'AB01', 1: 'AB02', 2: 'AB03', 3: 'AB04', 4: 'AB05', 5: 'AB06', 6: 'AB07', 7: 'AB081', 8: 'AB082', 9: 'AB083', 10: 'AB09', 11: 'AB10', 12: 'AB11', 13: 'AB12', 14: 'AB13', 15: 'AB14', 16: 'AB15', 17: 'AB16', 18: 'AB17', 19: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.11626024,  0.15274723,  0.12843712, ...,  0.36707819,
         0.30821529,  0.18607783],
       [ 0.15288359,  0.79525858, -0.12393902, ...,  0.95127606,
         0.51105404, -0.0086262 ],
       [-0.10803592,  0.22257306,  0.06777478, ...,  0.43193549,
         0.5803296 , -0.0854029 ],
       ...,
       [ 0.18977034,  0.06333469,  0.03730256, ...,  0.06692554,
        -0.1842189 ,  0.34666154],
       [ 0.46121225, -0.05615357, -0.04682136, ...,  0.33846882,
        -0.07609218,  0.15525247],
       [ 0.72029489,  0.13755193, -0.19765973, ..., -0.12733953,
         0.13950531, -0.09590579]])

In [10]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.116260,0.152747,0.128437,-0.168914,0.335952,-0.163506,0.289471,-0.164157,-0.112088,-0.215092,...,-0.169851,-0.232993,0.206899,0.081048,0.087011,-0.066176,0.384215,0.367078,0.308215,0.186078
1,0.152884,0.795259,-0.123939,-0.017086,0.057468,-0.173887,-0.051378,-0.090190,-0.182105,-0.200674,...,-0.150101,-0.179804,-0.033863,0.235776,-0.013047,-0.137038,0.175562,0.951276,0.511054,-0.008626
2,-0.108036,0.222573,0.067775,-0.089766,0.408120,-0.149850,0.461682,-0.207285,-0.163504,-0.101215,...,-0.108718,-0.183009,0.355673,-0.105422,0.328273,-0.160713,0.189675,0.431935,0.580330,-0.085403
3,-0.125039,0.211511,-0.111375,-0.057722,0.173165,-0.168334,0.138246,-0.192431,-0.193078,0.013967,...,-0.160995,-0.110600,0.134484,0.112681,0.059404,-0.021892,0.040524,0.448308,0.273445,-0.077635
4,-0.138636,0.282204,0.079704,-0.140916,0.242556,-0.070813,0.036644,-0.214607,-0.164184,-0.233235,...,-0.107751,-0.182548,0.400585,-0.085239,0.082974,0.433801,0.171166,0.015500,-0.070414,-0.050668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,0.842265,-0.027261,-0.087388,0.621318,-0.032711,1.400617,-0.088790,0.064123,0.041764,0.075889,...,0.746690,0.370330,1.187810,0.032654,0.285755,0.191604,-0.146465,-0.075858,-0.052954,0.211670
1324,-0.141186,-0.144963,-0.104052,-0.095101,-0.098736,1.153870,-0.183229,0.007238,-0.164956,0.528242,...,0.509736,1.042801,0.886454,0.154516,-0.015915,-0.024125,-0.143352,-0.140889,-0.179928,0.278095
1325,0.189770,0.063335,0.037303,0.396096,0.215092,0.905744,-0.127738,0.300558,-0.108303,0.037657,...,0.221734,0.117664,0.823608,0.300957,0.365588,-0.122770,-0.168415,0.066926,-0.184219,0.346662
1326,0.461212,-0.056154,-0.046821,0.539387,0.137011,1.332201,-0.053079,0.014405,0.112205,0.189531,...,0.738416,0.663533,0.474897,0.042114,0.173407,-0.162668,-0.136076,0.338469,-0.076092,0.155252


In [11]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class_New'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class_New'] = d

In [12]:
a = dataframe['Case'].tolist()
dataset['Case'] = a 
b = dataframe['15AB'].tolist()
dataset['15AB'] = b

In [15]:
os.chdir( '/home/yupaporn/CSV_file/')
testdf = dataset
testdf.to_csv('FVTest_20Class.csv')